In [1]:
import os
import json
import logging
import sys
import gzip
import os
import tarfile
import xml.etree.ElementTree as ET
import re
import random

from llama_index.core import (
    VectorStoreIndex,
    Settings,
    Document,
    StorageContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_classic.chains import create_retrieval_chain
from langchain_core.retrievers import BaseRetriever
from langchain_core.documents import Document as LangChainDocument
from langchain_core.prompts import PromptTemplate
from typing import List, Any
import gradio as gr

from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain

from langchain_openai import ChatOpenAI
from typing import Any, Dict, List, Tuple

from pathlib import Path

ModuleNotFoundError: No module named 'llama_index'

In [3]:
# --- 1. CONFIGURATION & IMPORTS ---
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Install langchain and its dependencies if not already installed
!pip install -qU langchain langchain-community

In [ ]:
# Set parameters
# Local path
TAR_PATH = (
     r""
)
# # e.g. r"C:\Users\user\Corpus\Conllu_output"
# OUTPUT_DIR = (
#     r"./Output"
# )

# Google API key: https://aistudio.google.com/api-keys
api_key = ""

# Define where you want to save the indexed data
PERSIST_DIR = "./storage_local_llama" 
ROOT_DIR =  Path.cwd().parent

input_questions_path = ROOT_DIR / "data" /"questions_20.json"
output_answers_path = ROOT_DIR / "data" /"llm_20.json"

DOC_LIMIT = 25
SOFA_NAMESPACE = "{http:///uima/cas.ecore}Sofa"

In [ ]:
# Set storage for Q&A
CHAT_LOG = [] 
LOG_FILE = "chat_history_llm.json" #

def save_log_to_disk():
    """Saves the current CHAT_LOG to a JSON file"""
    with open(LOG_FILE, "w", encoding="utf-8") as f:
        json.dump(CHAT_LOG, f, ensure_ascii=False, indent=2)
    print(f"Saved {len(CHAT_LOG)} interactions to {LOG_FILE}")

# Setup Models
print("Loading Embedding Model")
Settings.embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large")
Settings.node_parser = SentenceSplitter(chunk_size=1024, chunk_overlap=100)


# Load Data
def extract_raw_text(tar_path, doc_limit):
    """
    Iterates through the .xmi files in the tar archive and extracts the
    raw text content using ElementTree.
    """
    docs = []

    try:
        with tarfile.open(tar_path, "r:*") as tar:
            for m in tar.getmembers():
                name = m.name.lower()

                # Filter for XMI files
                if name.endswith((".xmi", ".xmi.gz", ".xmi.xmi.gz", ".xmi.xmi.gz")):
                    f = tar.extractfile(m)
                    if f is None:
                        continue

                    data = f.read()

                    if name.endswith(".gz"):
                        data = gzip.decompress(data)

                    # API for parsing and creating XML data
                    root = ET.fromstring(data)

                    # find sofa element with the text
                    sofa = root.find(f".//{SOFA_NAMESPACE}")

                    if sofa is not None:
                        text = sofa.get("sofaString")
                        if text:
                            # clean up unwanted characters before preprocessing
                            text = text.replace('\r\n', ' ').replace('\n', ' ').strip()
                            docs.append({
                                "id": os.path.basename(m.name).replace(".xmi", ""),
                                "text": text
                            })

                    # Stop after reaching the defined limit
                    if len(docs) >= doc_limit:
                        print(f"Reached document limit of {doc_limit}")
                        break

    except tarfile.TarError as e:
        print(f"Error reading tar file: {e}")
        return []
    except ET.ParseError as e:
        print(f"Error parsing XMI content: {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred during extraction: {e}")
        return []

    print(f"Successfully extracted raw text from {len(docs)} documents.")
    return docs

raw_data = extract_raw_text(TAR_PATH, DOC_LIMIT)
if not raw_data:
    print("ERROR: No documents were found! Check your TAR_PATH and XML Namespace.")
    sys.exit(1)


print("Converting dictionaries to Documents")
documents = []
for entry in raw_data:
    doc = Document(
        text=entry["text"],
        id_=entry["id"],
        metadata={
            "id_": entry["id"]}
    )
    documents.append(doc)


document_ids = [doc.id_ for doc in documents]

print(document_ids)



Loading Embedding Model
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/multilingual-e5-large
Load pretrained SentenceTransformer: intfloat/multilingual-e5-large
Load pretrained SentenceTransformer: intfloat/multilingual-e5-large
Successfully extracted raw text from 10 documents.
Converting dictionaries to Documents
Creating new index


Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/366 [00:00<?, ?it/s]

In [ ]:

# Indexing
if os.path.exists(PERSIST_DIR):
    print("Loading index from storage")
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)
else:
    print("Creating new index")
    index = VectorStoreIndex.from_documents(documents, show_progress=True)
    index.storage_context.persist(persist_dir=PERSIST_DIR)


# Adapter Class
class LlamaIndexToLangChainRetriever(BaseRetriever):
    llama_retriever: Any
    def _get_relevant_documents(self, query: str, *, run_manager=None) -> List[LangChainDocument]:
        nodes = self.llama_retriever.retrieve(query)
        langchain_docs = []
        for node in nodes:
            langchain_docs.append(
                LangChainDocument(page_content=node.get_content(), metadata=node.metadata)
            )
        return langchain_docs

In [ ]:
# Connect to Langchain
raw_retriever = index.as_retriever(similarity_top_k=5)
llama_retriever = LlamaIndexToLangChainRetriever(llama_retriever=raw_retriever)


"""
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=api_key,
    temperature=0,
    convert_system_message_to_human=True
)
"""

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    api_key=api_key,
    temperature=0,
    max_tokens=200
)

# Create Prompt
custom_template = """
    Du bist ein strikt regelbasierter Assistent für Parlamentsprotokolle.
    
    ZULÄSSIGE INFORMATIONSQUELLE:
    - Ausschließlich der unten bereitgestellte Kontext aus den Dokumenten.
    
    VERBOTEN:
    - Nutzung von Vorwissen, Weltwissen oder Annahmen.
    - Ableitungen, Interpretationen oder Ergänzungen über den Kontext hinaus.
    - Vermischung mehrerer Dokumente, wenn die Information nicht explizit enthalten ist.
    
    PFLICHTREGEL:
    Wenn die Antwort nicht wortgleich oder eindeutig aus dem Kontext hervorgeht, antworte exakt mit:
    "Die Information ist im Kontext nicht enthalten."
    
    ANTWORTFORMAT:
    - Kurz und präzise.
    - Nur die zur Frage notwendigen Fakten.
    - Keine Einleitung, keine Zusammenfassung, keine Bewertung.
    - Aufzählungspunkte nur, wenn mehrere Fakten explizit genannt sind.
    - Keine zusätzlichen Erklärungen.
    
    KONTEXT (einzige Wissensbasis):
    {context}
    
    FRAGE:
    {input}
    
    ANTWORT:
    """

PROMPT = ChatPromptTemplate.from_template(custom_template)

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=PROMPT
)

qa_chain = create_retrieval_chain(
    retriever=llama_retriever,
    combine_docs_chain=document_chain
)

In [ ]:
def load_questions(input_path: str) -> List[Dict[str, Any]]:
    if not os.path.isfile(input_path):
        raise FileNotFoundError(f"Input file not found: {input_path}")

    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    if not isinstance(data, list):
        raise ValueError("Input JSON must be a list of objects (array).")

    for i, item in enumerate(data):
        if not isinstance(item, dict):
            raise ValueError(f"Item at index {i} is not an object.")
        if "question" not in item:
            raise ValueError(f"Item at index {i} has no 'question' field.")
        if not isinstance(item["question"], str) or not item["question"].strip():
            raise ValueError(f"Item at index {i} has empty/invalid 'question' field.")

    return data


def save_output(output_path: str, data: List[Dict[str, Any]]) -> None:
    out_dir = os.path.dirname(os.path.abspath(output_path))
    os.makedirs(out_dir, exist_ok=True)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)


def ask_rag_only_question(qa_chain, question_text: str, snippet_len: int = 300):
    response = qa_chain.invoke({"input": question_text})

    answer_text = response.get("answer", "")
    if not isinstance(answer_text, str):
        answer_text = str(answer_text)
    answer_text = answer_text.strip()

    source_ids = []
    context_snippets = []

    docs = response.get("context", None)
    if docs is None:
        docs = response.get("documents", [])

    for doc in docs:
        s_id = None
        try:
            s_id = doc.metadata.get("id_")
        except Exception:
            s_id = None

        text = getattr(doc, "page_content", "") or ""

        if isinstance(s_id, str) and s_id:
            source_ids.append(s_id)
            snippet = text[:snippet_len].replace("\n", " ").strip()
            context_snippets.append({"source_id": s_id, "snippet": snippet})

    source_ids = list(dict.fromkeys(source_ids))
    return answer_text, source_ids, context_snippets


def process_questions_file(qa_chain, input_path: str, output_path: str) -> None:
    items = load_questions(input_path)

    for idx, item in enumerate(items, start=1):
        question_text = item["question"].strip()

        answer_text = ""
        answer_source_ids = []
        context_snippets = []

        try:
            answer_text, answer_source_ids, context_snippets = ask_rag_only_question(
                qa_chain, question_text
            )
        except Exception as e:
            answer_text = f"Error: {str(e)}"
            answer_source_ids = []
            context_snippets = []

        item["answer"] = answer_text
        item["answer_source_id"] = answer_source_ids
        item["answer_context_snippets"] = context_snippets

        print(f"[{idx}/{len(items)}] id={item.get('id', 'NA')} done")

    save_output(output_path, items)
    print(f"Saved results to {output_path}")

# input_questions_path = "data/questions.json"
# output_answers_path = "data/output.json"

process_questions_file(qa_chain, input_questions_path, output_answers_path)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[1/12] id=1 done
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[2/12] id=2 done
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[3/12] id=3 done
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https: